# Project 3


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="moviegenre.png"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).

• The project must be carried out in the groups assigned for module 4.
• Use clear and rigorous procedures.
• The delivery of the project is on July 12, 2020, 11:59 pm, through Sicua + (Upload: the API and the report in PDF format).
• No projects will be received after the delivery time or by any other means than the one established. 




### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [3]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [4]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [5]:
%%time

import re
def pre_process(text):
    # lowercase
    text = text.lower()
    # tags
    text = re.sub('&lt;/?.*?&gt;',' &lt;&gt; ',text)
    # special characters and digits
    text=re.sub('(\\d|\\W)+',' ',text)
    # remove punctuation
    #text = re.sub('[.;:!\'?,\"()\[\]]', '', text)
    #text = [REPLACE.sub('', line) for line in text]
    
    return text
dataTraining['plot_low']=dataTraining['plot'].apply(lambda x:pre_process(x))

import nltk
nltk.corpus.stopwords.words('english')
nltk.download('wordnet') 
from nltk.corpus import stopwords


english_stop_words=stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

dataTraining['plot_low_rm'] = remove_stop_words(dataTraining['plot_low'])


from nltk.stem.porter import PorterStemmer
def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

dataTraining['plot_low_rm_stem'] = get_stemmed_text(dataTraining['plot_low_rm'])


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Wall time: 12.7 s


In [6]:
dataTraining[['plot','plot_low','plot_low_rm','plot_low_rm_stem']]

,plot,plot_low,plot_low_rm,plot_low_rm_stem
3107,most is the story of a single father who takes...,most is the story of a single father who takes...,story single father takes eight year old son w...,stori singl father take eight year old son wor...
900,a serial killer decides to teach the secrets o...,a serial killer decides to teach the secrets o...,serial killer decides teach secrets satisfying...,serial killer decid teach secret satisfi caree...
6724,"in sweden , a female blackmailer with a disfi...",in sweden a female blackmailer with a disfigur...,sweden female blackmailer disfiguring facial s...,sweden femal blackmail disfigur facial scar me...
4704,"in a friday afternoon in new york , the presi...",in a friday afternoon in new york the presiden...,friday afternoon new york president tredway co...,friday afternoon new york presid tredway corpo...
2582,"in los angeles , the editor of a publishing h...",in los angeles the editor of a publishing hous...,los angeles editor publishing house carol hunn...,lo angel editor publish hous carol hunnicut go...
...,...,...,...,...
8417,""" our marriage , their wedding . "" it ' s l...",our marriage their wedding it s lesson number...,marriage wedding lesson number one newly engag...,marriag wed lesson number one newli engag coup...
1592,"the wandering barbarian , conan , alongside ...",the wandering barbarian conan alongside his go...,wandering barbarian conan alongside goofy rogu...,wander barbarian conan alongsid goofi rogu pal...
1723,"like a tale spun by scheherazade , kismet fol...",like a tale spun by scheherazade kismet follow...,like tale spun scheherazade kismet follows rem...,like tale spun scheherazad kismet follow remar...
7605,"mrs . brisby , a widowed mouse , lives in a...",mrs brisby a widowed mouse lives in a cinder b...,mrs brisby widowed mouse lives cinder block ch...,mr brisbi widow mous live cinder block childre...


### Create count vectorizer


In [7]:
vect = CountVectorizer(ngram_range=(1,2),lowercase=True,max_features=10000)

X_dtm = vect.fit_transform(dataTraining['plot_low_rm_stem'])
X_dtm.shape

(7895, 10000)

In [26]:
X_dtm.shape[1]

10000

In [8]:
print(vect.get_feature_names()[:50])

['aaron', 'abandon', 'abbey', 'abbi', 'abbott', 'abduct', 'abe', 'abel', 'abigail', 'abil', 'abl', 'abl convinc', 'abl escap', 'abl find', 'abl get', 'abl make', 'abner', 'aboard', 'aboard ship', 'abort', 'abound', 'abraham', 'abraham lincoln', 'abroad', 'abruptli', 'absenc', 'absent', 'absolut', 'absorb', 'absurd', 'abu', 'abus', 'abus husband', 'academ', 'academi', 'academi award', 'accept', 'accept invit', 'accept job', 'accept offer', 'access', 'accid', 'accident', 'accident death', 'accident kill', 'acclaim', 'accommod', 'accompani', 'accomplic', 'accomplish']


### Create y

In [9]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [24]:
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

# Red Neuronal

In [27]:
model = Sequential()
model.add(Embedding(X_dtm.shape[1] + 1, 128)
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

SyntaxError: invalid syntax (<ipython-input-27-f8eb61c16327>, line 3)

In [ ]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train_genres, validation_data=[X_test, y_test_genres], 
          batch_size=128, epochs=10, verbose=1)
          #callbacks=[PlotLossesKeras()])

In [ ]:
from sklearn.metrics import accuracy_score
pred = model.predict_classes(X_test)
accuracy_score(pred, y_test_genres)